In [ ]:
%run "./Setup.ipynb"

In [ ]:
spark

In [ ]:
inputPath = "E:\\PySpark\\data\\users.json"

In [ ]:
from pyspark.sql.functions import *

In [ ]:
spark.conf.get("spark.sql.warehouse.dir")

In [ ]:
df1 = spark.read.json(inputPath)
df1.show()

In [ ]:
df1.createOrReplaceTempView("users")

In [ ]:
spark.catalog.currentCatalog()

In [ ]:
spark.catalog.listTables()

In [ ]:
qry = """select age, count(1) as count
        from users
        where age is not null
        group by age
        order by count
        limit 5"""

df2 = spark.sql(qry)
df2.show()

In [ ]:
spark2 = spark.newSession()

In [ ]:
df2.createOrReplaceTempView("users2")

In [ ]:
spark2.catalog.listTables()

In [ ]:
spark2.catalog.dropTempView("users")

In [ ]:
spark2.catalog.listTables()

In [ ]:
qry = """select age, count(1) as count
        from users2
        where age is not null
        group by age
        order by count
        limit 5"""

spark2.sql(qry).show()

In [ ]:
spark.sql(qry).show()

### Global Temp View

In [ ]:
df1.createGlobalTempView("gusers")

In [ ]:
spark.catalog.listTables('global_temp')

In [ ]:
qry = """select age, count(1) as count
        from global_temp.gusers
        where age is not null
        group by age
        order by count
        limit 5"""

In [ ]:
spark.sql(qry).show()

In [ ]:
spark2.sql(qry).show()

### Tables

In [ ]:
df3 = df1.where("age is not null")
df3.show()

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
df3.rdd.getNumPartitions()

In [ ]:
df3.write.format("json").saveAsTable("users_json")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("select * from users_json where age > 25").show()

In [ ]:
#df3.write.format("json").mode("append").saveAsTable("users_json")
df3.write.format("json").mode("overwrite").saveAsTable("users_json")

In [ ]:
spark.sql("select * from users_json where age > 25").show()

In [ ]:
df3.write.format("csv").option("header", True).saveAsTable("users_csv")

In [ ]:
df3.write.format("parquet").saveAsTable("users_prquet")

In [ ]:
spark.sql("create table users_tv using json as select * from users")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("drop table users_tv")

#### External Tables

In [ ]:
df3.write.format("json").option("path", "E:\\PySpark\\external\\users_json_ext").saveAsTable("users_json_ext")

In [ ]:
spark.catalog.listTables()

#### Partitioned Tables

In [ ]:
df3.show()

In [ ]:
df3.write.format("json").partitionBy("gender").saveAsTable("users_part")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("select * from users_part").show()

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("drop table users_csv")

In [ ]:
spark.sql("select * from users_csv").show()

In [ ]:
spark.sql("drop table users_json_ext")

In [ ]:
spark.sql("select * from users_json_ext").show()